In [1]:
import pandas as pd
import numpy as np
import docplex
from docplex.mp.model import Model
from docplex.mp.progress import *
import time
import sys
import math
from sklearn.model_selection import train_test_split
import math

In [2]:
def DataPreperation(datasetname,sampleAmount, featCART, seed):
    global numericalGroup
    numericalGroup = []
    global data
    global trainData
    global samplePos
    global trainDataDummies
    global groupsOfFeatures
    global featureToGroup
    global testData
    global testDataDummies
    global samplePosTest
    # Read in prefered dataset
    if(datasetname == 'chess'):
        dataChess = pd.read_csv("kr-vs-kp.data", header=0, names=['bkblk','bknwy','bkon8','bkona','bkspr','bkxbq','bkxcr','bkxwp','blxwp','bxqsq','cntxt','dsopp','dwipd','hdchk','katri','mulch','qxmsq','r2ar8','reskd','reskr','rimmx','rkxwp','rxmsq','simpl','skach','skewr','skrxp','spcop','stlmt','thrsk','wkcti','wkna8','wknck','wkovl','wkpos','wtoeg','white_win'])
        if featCART:
            dataChess = dataChess[['white_win','rimmx','wknck','bxqsq','wkna8', 'wkpos','bkxbq','katri', 'bkblk']]
        data, testData = train_test_split(dataChess.dropna(), train_size=sampleAmount, random_state=seed)
        trainData = data.drop(columns=['white_win'])
        samplePos = (data['white_win'] == 'won')*1
        samplePosTest = (testData['white_win'] == 'won')*1
        testData = testData.drop(columns='white_win')
    elif(datasetname == 'adult'):
        dataAdult = pd.read_csv("a1a.txt", delim_whitespace=True, header=None, names = ['salery50k','age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country']) 
        if featCART:
            dataAdult = dataAdult[['salery50k','relationship', 'occupation', 'education', 'capital-gain','workclass']]
        if sampleAmount > len(dataAdult.dropna()):
            sampleAmount = math.floor(0.9*len(dataAdult.dropna()))
        data, testData = train_test_split(dataAdult.dropna(), train_size=sampleAmount, random_state=seed)
        trainData = data.drop(columns='salery50k')
        samplePos = (data['salery50k'] == 1)*1
        
        samplePosTest = (testData['salery50k'] == 1)*1
        testData = testData.drop(columns='salery50k')
        
        numericalGroup = [1, 3, 5, 11, 12, 13]
    elif(datasetname == 'breast'):
        dataBreast = pd.read_csv("breast.txt", delim_whitespace=True, header=0, names=['Class','Sample code number','Clump Thickness','Uniformity of Cell Size','Uniformity of Cell Shape','Marginal Adhesion','Single Epithelial Cell Size','Bare Nuclei','Bland Chromatin','Normal Nucleoli','Mitoses'])
        dataBreast = dataBreast.drop(columns='Sample code number') #drop id-numbers of observations
        if featCART:
            dataBreast = dataBreast[['Class','Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Bare Nuclei']]
        data, testData = train_test_split(dataBreast.dropna(), train_size=sampleAmount, random_state=seed)
        
        #for i in trainData.columns: #take away the rownumber: of all variables.
        #        trainData[i] = trainData[i].str[trainData.iloc[0][i].index(':')+1:]
        samplePos = (data['Class'] == 2)*1
        trainData = data.drop(columns='Class')
        
        samplePosTest = (testData['Class'] == 2)*1
        testData = testData.drop(columns='Class')
        
        numericalGroup = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    elif(datasetname == 'mushroom'):
        dataMushroom = pd.read_csv("agaricus-lepiota.data", header=0, names=['edible','cap-shape','cap-surface','cap-color','bruises?','odor','gill-attachment','gill-spacing','gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring','veil-type','veil-color','ring-number','ring-type','spore-print-color','population','habitat'])
        dataMushroom = dataMushroom.replace("?", np.nan)
        if featCART:
            dataMushroom = dataMushroom[['edible','odor', 'spore-print-color']]
            
        data, testData = train_test_split(dataMushroom.dropna(), train_size=sampleAmount, random_state=seed)
        
        #data = dataMushroom.dropna().sample(n=5, random_state=1).drop(columns=['cap-color','bruises?','odor','gill-attachment','gill-spacing','gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring','veil-type','veil-color','ring-number','ring-type','spore-print-color','population','habitat'])

        trainData = data.drop(columns='edible')
        samplePos = (data['edible'] == 'e')*1
        samplePosTest = (testData['edible'] == 'e')*1
        testData = testData.drop(columns='edible')
    elif(datasetname == 'tictactoe'):
        dataTicTac = pd.read_csv("tic-tac-toe.data", header=0, names=['top-left-square','top-middle-square','top-right-square','middle-left-square','middle-middle-square','middle-right-square','bottom-left-square','bottom-middle-square','bottom-right-square','Class'])
        if featCART:
            dataTicTac = dataTicTac[['Class','middle-middle-square','bottom-right-square','top-left-square','middle-right-square','middle-left-square','top-right-square','bottom-left-square','bottom-middle-square']]
        data, testData = train_test_split(dataTicTac.dropna(), train_size=sampleAmount, random_state=seed)
        
        samplePos = (data['Class'] == 'positive')*1
        trainData = data.drop(columns=['Class'])
        samplePosTest = (testData['Class'] == 'positive')*1
        testData = testData.drop(columns=['Class'])
    elif(datasetname == 'monks'):
        dataMonks = pd.read_csv("monks-1.test", header=0, sep=' ', names=['empty','class','a1','a2','a3','a4','a5','a6','Id'])
        dataMonks = dataMonks.drop(columns=['empty', 'Id'])
        if featCART:
            dataMonks = dataMonks[['class','a1', 'a2', 'a4', 'a5']]
        for i in dataMonks.columns:
            dataMonks[i] = dataMonks[i].apply(str)
        if(sampleAmount > len(dataMonks.dropna())):
            sampleAmount = math.floor(len(dataMonks)*0.9)
        data, testData = train_test_split(dataMonks.dropna(), train_size=sampleAmount, random_state=seed)

        samplePos = (data['class'] == '1')*1
        trainData = data.drop(columns=['class'])
        samplePosTest = (testData['class'] == '1')*1
        testData = testData.drop(columns=['class'])
    elif(datasetname == 'votes'):
        dataVotes = pd.read_csv("house-votes-84.data", header=None, names =['Class','handicapped-infants','water-project-cost-sharing','adoption-of-the-budget-resolution','physician-fee-freeze','el-salvador-aid','religious-groups-in-schools','anti-satellite-test-ban','aid-to-nicaraguan-contras','mx-missile','immigration','synfuels-corporation-cutback','education-spending','superfund-right-to-sue','crime','duty-free-exports','export-administration-act-south-africa'])
        dataVotes = dataVotes.replace("?", np.nan)
        if featCART:
            dataVotes = dataVotes[['Class','physician-fee-freeze']]
        if(sampleAmount > len(dataVotes.dropna())):
            sampleAmount = math.floor(len(dataVotes.dropna())*0.9)
        data, testData = train_test_split(dataVotes.dropna(), train_size=sampleAmount, random_state=seed)

        samplePos = (data['Class'] == 'democrat')*1
        trainData = data.drop(columns='Class')
        samplePosTest = (testData['Class'] == 'democrat')*1
        testData = testData.drop(columns='Class')
    elif(datasetname == 'heart'):
        dataHeart1 = pd.read_csv("SPECT.test", header=None)
        dataHeart2 = pd.read_csv("SPECT.train", header=None)
        frames = [dataHeart1, dataHeart2]
        dataHeart = pd.concat(frames)
        for i in dataHeart.columns:
            dataHeart[i] = dataHeart[i].apply(str)
        dataHeart.columns = ['OVERALL_DIAGNOSIS','F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13','F14','F15','F16','F17','F18','F19','F20','F21','F22']
        if featCART:
            dataHeart = dataHeart[['OVERALL_DIAGNOSIS','F13','F11','F16','F10','F22','F20']]
        if (sampleAmount > len(dataHeart.dropna())):
            sampleAmount = math.floor(len(dataHeart.dropna())*0.9)
        data, testData = train_test_split(dataHeart.dropna(), train_size=sampleAmount, random_state=seed)
        
        samplePos = (data['OVERALL_DIAGNOSIS'] == '1')*1
        trainData = data.drop(columns='OVERALL_DIAGNOSIS')
        samplePosTest = (testData['OVERALL_DIAGNOSIS'] == '1')*1
        testData = testData.drop(columns='OVERALL_DIAGNOSIS')
    elif(datasetname == 'student'):
        studentMat = pd.read_csv("student-mat.csv", sep=";")
        studentMat['nonAlcoholic'] = np.where((studentMat['Dalc'] > 1) & (studentMat['Walc'] > 1), 0, 1)
        dataStudent = studentMat.drop(columns=['Walc', 'Dalc'])
        if featCART:
            dataStudent = dataStudent[['nonAlcoholic','goout','famsize','reason','famsup','studytime','Mjob','activities','sex','famrel','G1']]
        if (sampleAmount > len(dataStudent.dropna())):
            sampleAmount = math.floor(395*0.9)
        data, testData = train_test_split(dataStudent.dropna(), train_size=sampleAmount, random_state=seed)
        
        trainData = data.drop(columns='nonAlcoholic')
        samplePos = (data['nonAlcoholic'] == 1)*1    
        
        samplePosTest = (testData['nonAlcoholic'] == 1)*1
        testData = testData.drop(columns='nonAlcoholic')
        numericalGroup = [3, 7, 8, 13, 14, 15, 24, 25, 26, 27, 28]
    elif(datasetname == 'heloc'):
        dataHeloc = pd.read_csv("heloc_dataset_v1.csv", sep=",")
        if featCART:
            dataHeloc = dataHeloc[['RiskPerformance','ExternalRiskEstimate', 'MSinceMostRecentInqexcl7days']]
        for i in dataHeloc.columns:
            if i != 'RiskPerformance':
                dataHeloc[i] = pd.qcut(dataHeloc[i], q=10, duplicates='drop')

        data, testData = train_test_split(dataHeloc.dropna(), train_size=sampleAmount, random_state=seed)
        trainData = data.drop(columns='RiskPerformance')
        samplePos = (data['RiskPerformance'] == 'Good')*1
        samplePosTest = (testData['RiskPerformance'] == 'Good')*1
        testData = testData.drop(columns='RiskPerformance')
        numericalGroup = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
    else:
        print('ERROR: dataset name not found')
    #process data for tree
    if(datasetname == 'heart'):
        trainDataDummies = pd.get_dummies(trainData)
        testDataDummies = pd.get_dummies(testData)
    else:
        start = 0
        for i in trainData.columns:
            if (start == 0):
                trainDataDummiesTemp = pd.get_dummies(trainData[i], prefix=i)
                trainDataDummies = trainDataDummiesTemp[sorted(trainDataDummiesTemp.columns.tolist(), key=str.lower)]
                start = 1
            else:
                trainDataDummiesTemp = pd.get_dummies(trainData[i], prefix=i)
                trainDataDummies = trainDataDummies.join(trainDataDummiesTemp[sorted(trainDataDummiesTemp.columns.tolist(), key=str.lower)])
        trainDataDummies = trainDataDummies.loc[:, (trainDataDummies != 0).any(axis=0)]

        start = 0
        for i in testData.columns:
            if (start == 0):
                testDataDummiesTemp = pd.get_dummies(testData[i], prefix=i)
                testDataDummies = testDataDummiesTemp[sorted(testDataDummiesTemp.columns.tolist(), key=str.lower)]
                start = 1
            else:
                testDataDummiesTemp = pd.get_dummies(testData[i], prefix=i)
                testDataDummies = testDataDummies.join(testDataDummiesTemp[sorted(testDataDummiesTemp.columns.tolist(), key=str.lower)])
        testDataDummies = testDataDummies.loc[:, (testDataDummies != 0).any(axis=0)]
    
    groupsN = trainData.shape[1]
    G = range(1, groupsN + 1)
    groupsOfFeatures = trainData.nunique()
    featureToGroup = []
    for g in G:
        amountOfFeatures = groupsOfFeatures.iloc[g-1]
        for f in range(amountOfFeatures):
            featureToGroup.append(g)

In [3]:
def decisionTree(typeOfTree, C, ExclSameBranchFollowing, strength, anchor, relax, numerical, combCon, maxCard, sensOn, specOn, sensitivity, specificity):
    model = Model("tree")
    model.parameters.randomseed = 42
    
    #datashape
    N = trainData.shape[0] # amount of samples
    I = range(1, N + 1) # all samples
    groupsN = trainData.shape[1] # amount of groupes
    groupsOfFeatures = trainData.nunique() # amount of features per group
    G = range(1, groupsN + 1) #groups
    J = range(1, len(trainDataDummies.columns) + 1) #features
    Iplus = np.where(samplePos == 1)[0]
    Iplus = [x+1 for x in Iplus]
    Imin = np.where(samplePos == 0)[0]
    Imin = [x+1 for x in Imin]
    
    
    #tree topology
    if (typeOfTree == 1):
        #simple tree of depth 2
        dNodesN = 3
        lNodesN = 4
        Bplus = [1, 3]
        Bmin = [2, 4]
        route = np.array([[1, 1, 0], [1, -1, 0], [-1, 0, 1], [-1, 0, -1]])
        nonAdjLeaf = [1]
        nonAdjLeafSymm = [1]
        AdjLeaf = [2, 3]
    if(typeOfTree == 2):
        #binary tree with depth 2,5
        dNodesN = 5
        lNodesN = 6
        Bplus = [1, 3, 5]
        Bmin = [2, 4, 6]
        route = np.array([[1, 1, 1, 0, 0], [1, 1, -1, 0, 0], [1, -1, 0, 1, 0], \
                          [1, -1, 0, -1, 0], [-1, 0, 0, 0, 1],  [-1, 0, 0, 0, -1]])
        nonAdjLeaf = [1, 2]
        nonAdjLeafSymm = [2]
        AdjLeaf = [3, 4, 5]
    if(typeOfTree == 3):
        #binary tree with depth 3
        dNodesN = 7
        lNodesN = 8
        Bplus = [1, 3, 5, 7]
        Bmin = [2, 4, 6, 8]
        route = np.array([[1, 1, 1, 0, 0, 0, 0], [1, 1, -1, 0, 0, 0, 0], [1, -1, 0, 1, 0, 0, 0], \
                         [1, -1, 0, -1, 0, 0, 0], [-1, 0, 0, 0, 1, 1, 0], [-1, 0, 0, 0, 1, -1, 0], \
                         [-1, 0, 0, 0, -1, 0, 1], [-1, 0, 0, 0, -1, 0, -1]])
        nonAdjLeaf = [1, 2, 5]
        nonAdjLeafSymm = [1, 2, 5]
        AdjLeaf = [3, 4, 6, 7]
    if(typeOfTree == 4):
        #binary tree with depth 3,5
        dNodesN = 7
        lNodesN = 8
        Bplus = [1, 3, 5, 7]
        Bmin = [2, 4, 6, 8]
        route = np.array([[1, 1, 1, 1, 0, 0, 0], [1, 1, 1, -1, 0, 0, 0], [1, 1, -1, 0, 1, 0, 0], \
                         [1, 1, -1, 0, -1, 0, 0], [1, -1, 0, 0, 0, 1, 0], [1, -1, 0, 0, 0, -1, 0], \
                         [-1, 0, 0, 0, 0, 0, 1], [-1, 0, 0, 0, 0, 0, -1]])
        nonAdjLeaf = [1, 2, 3]
        nonAdjLeafSymm = [3]
        AdjLeaf = [4, 5, 6, 7]
        
    #make nodes
    K = range(1, dNodesN + 1) #decision nodes
    B = range(1, lNodesN + 1) #leaf nodes

    # --- decision variables --- 
    idv = [(g, k) for g in G for k in K]
    if(relax): 
        v = model.continuous_var_dict(keys=idv, lb=0, ub=1, name="V") #relaxation of variables of paragraph 4.3.4
    else:
        v = model.binary_var_dict(keys=idv, name="V")
    idz = [(j,k) for j in J for k in K]

    if(relax):
        idzAdjLeaf = [(j,k) for j in J for k in AdjLeaf]
        idzNonAdjLeaf = [(j,k) for j in J for k in nonAdjLeaf]
        z = model.continuous_var_dict(keys=idzAdjLeaf, lb=0, ub=1, name="Z")
        zNonAdjLeaf = model.binary_var_dict(keys=idzNonAdjLeaf, name="Z")
        z.update(zNonAdjLeaf)
    else:
        idz = [(j,k) for j in J for k in K]
        z = model.binary_var_dict(keys=idz, name="Z")
    idc = [(b,i) for b in B for i in I]
    if(relax):
        c = model.continuous_var_dict(keys=idc,ub=1, name="C")
    else:
        c = model.binary_var_dict(keys=idc, name="C")
    idL = [(i,k) for i in I for k in K]
    L = model.binary_var_dict(keys=idL, name="L")
    idR = [(i,k) for i in I for k in K]
    R = model.binary_var_dict(keys=idL, name="R")
    if(numerical):
        idw = [(g,k) for k in K for g in numericalGroup]
        w = model.binary_var_dict(keys=idw, name="W")
    if(combCon):
        if(numerical):
            idw = [(g,k) for k in K for g in G if ((g not in numericalGroup) and (groupsOfFeatures.iloc[g-1] > maxCard))]
            wcomb = model.binary_var_dict(keys=idw, name="W")
            w.update(wcomb)
        else:
            idw = [(g,k) for k in K for g in G if ((g not in numericalGroup) and (groupsOfFeatures.iloc[g-1] > maxCard))]
            w = model.binary_var_dict(keys=idw, name="W")

    # --- constraints ---
    for k in K:
        model.add_constraint(model.sum(v[g, k] for g in G) == 1) #constraint (1)

    for k in K:
        for j in J:
            model.add_constraint(z[j,k] <= v[featureToGroup[j-1], k]) #constraint (2)

    for k in K:
        for i in I:
            model.add_constraint(L[i,k] == sum(z[j,k] for j in J if trainDataDummies.iloc[i-1,j-1] == 1)) #constraint (3)
            model.add_constraint(R[i,k] == 1 - L[i,k]) #constraint (4)
    
    if(strength):
        for i in I:
            for k in K:
                model.add_constraint(sum(c[b,i] for b in B if route[b-1, k-1] == 1) <= L[i,k]) #constraint (11)
                model.add_constraint(sum(c[b,i] for b in B if route[b-1, k-1] == -1) <= R[i,k]) #constraint (12)
    else:
        for b in B:
            for i in I:
                for k in K:
                    switch = route[b-1, k-1]
                    if switch == 1:
                        model.add_constraint(c[b,i] <= L[i,k]) #constraint (5)
                    elif switch == -1:
                        model.add_constraint(c[b,i] <= R[i,k]) #constraint (6)
        for i in I:
            model.add_constraint(model.sum(c[b,i] for b in B) == 1) #constraint (7)
    
    if(anchor): 
        for g in G:
            for k in nonAdjLeafSymm: #only for nonleafs with symmatrical subtree
                model.add_constraint(z[featureToGroup.index(g) + 1, k] == v[g,k]) #constraint (13)
                        
    if(ExclSameBranchFollowing):
        #for k in K:
        test=1
            #add constraint (9) and (10)
            
    if(numerical):
        #numerical value constraint 4.4
        for k in K:
            for g in numericalGroup:
                featureList = [i+1 for i, x in enumerate(featureToGroup) if x == g]
                if (len(featureList) > 1):
                    for j in featureList:
                        if(j == featureList[0]):    
                            model.add_constraint(z[j,k] >= z[j+1,k] - w[g,k])
                        elif(j == featureList[-1]):
                            model.add_constraint(z[j,k] >= z[j-1,k] - (1 - w[g,k]))
                        else:
                            model.add_constraint(z[j,k] >= z[j+1,k] - w[g,k])
                            model.add_constraint(z[j,k] >= z[j-1,k] - (1 - w[g,k]))                     
                
    if(combCon):
        for k in K:
            for g in G:
                if((g not in numericalGroup) and (groupsOfFeatures.iloc[g-1] > maxCard)):
                    featureList = [i for i, x in enumerate(featureToGroup) if x == g]
                    model.add_constraint(model.sum(z[j+1,k] for j in featureList) <= maxCard + (groupsOfFeatures.iloc[g-1] - maxCard)*(1-w[g,k]))
                    model.add_constraint(model.sum(z[j+1,k] for j in featureList) >= (groupsOfFeatures.iloc[g-1] - maxCard) - (groupsOfFeatures.iloc[g-1] - maxCard)*w[g,k])
        
        
    
    # --- objective ---
    model.objectiveTruePos = model.sum(model.sum(c[b,i] for b in Bplus) for i in Iplus)
    model.objectiveTrueNeg = model.sum(model.sum(c[b,i] for b in Bmin) for i in Imin)
    
    if(sensOn):
        
        model.add_constraint(model.sum(model.sum(c[b,i] for b in Bplus) for i in Iplus) >= math.ceil((1-sensitivity)*len(Iplus) ))
        
        totalObjective = model.objectiveTrueNeg
        
        sensitivity
    elif(specOn):
        model.add_constraint(model.sum(model.sum(c[b,i] for b in Bmin) for i in Imin) >= math.ceil((1-specificity)*len(Imin) ))

        totalObjective = model.objectiveTruePos
    else:
        totalObjective = model.objectiveTruePos + C*model.objectiveTrueNeg 
    
    model.maximize(totalObjective)
    
    return model

In [4]:
def treeValidation(solution, testDataDummies, trainDataDummies, typeOfTree):
    global LikArray
    global dfTest
    global decNodeFeatureDoubleList
    error = False
    decNodeFeatureDoubleList = []
    decNodeFeatureList = []
    
    if(typeOfTree == 1):
        nodeAmount = 3
    elif(typeOfTree == 2):
        nodeAmount = 5
    elif(typeOfTree == 3):
        nodeAmount = 7
    elif(typeOfTree == 4):
        nodeAmount = 7
    for currentNode in range(1, nodeAmount + 1):
        for feature in range(1, len(trainDataDummies.columns) + 1):
                value = sol.get_value('Z_' + str(feature) + '_' + str(currentNode))
                if(value > 0):
                    decNodeFeatureList.append(feature)
        decNodeFeatureDoubleList.append(decNodeFeatureList)
        decNodeFeatureList = []
    

    dfTest = pd.DataFrame(columns=trainDataDummies.columns)

    for col in dfTest.columns:
        if col in testDataDummies.columns: # copy columns from testdata to dataframe
            dfTest[col] = testDataDummies[col]
    dfTest = dfTest.fillna(0) #works
    
    #first part done
    
    #now find good route
    LikArray = np.zeros((len(dfTest.index), len(decNodeFeatureDoubleList)))
    
    for i in range(len(dfTest)):
        for k in range(len(decNodeFeatureDoubleList)):
            #check if Lik equals 1 or 0 
                for feature in decNodeFeatureDoubleList[k]:
                    if(dfTest.iloc[i,feature-1] == 1):
                        LikArray[i,k] = 1
                        
    totalCorrectClassified = 0
    if(typeOfTree == 1):
        for i in range(len(dfTest)):
            if(LikArray[i,1-1] == 1):
                if(LikArray[i,2-1] == 1):
                    if(samplePosTest.iloc[i] == 1):
                        totalCorrectClassified += 1
                else:
                    if(samplePosTest.iloc[i] == 0):
                        totalCorrectClassified += 1
            else:
                if(LikArray[i,3-1] == 1):
                    if(samplePosTest.iloc[i] == 1):
                        totalCorrectClassified += 1
                else:
                    if(samplePosTest.iloc[i] == 0):
                        totalCorrectClassified += 1
                        
    elif(typeOfTree == 2):
        for i in range(len(dfTest)):
            if(LikArray[i,1-1] == 1):
                if(LikArray[i,2-1] == 1):
                    if(LikArray[i,3-1] == 1):
                        if(samplePosTest.iloc[i] == 1):
                            totalCorrectClassified += 1
                            
                    else:
                        if(samplePosTest.iloc[i] == 0):
                            totalCorrectClassified += 1
                            
                else:
                    if(LikArray[i,4-1] == 1):
                        if(samplePosTest.iloc[i] == 1):
                            totalCorrectClassified += 1
                            
                    else:
                        if(samplePosTest.iloc[i] == 0):
                            totalCorrectClassified += 1
                            
            else:
                if(LikArray[i,5-1] == 1):
                    if(samplePosTest.iloc[i] == 1):
                        totalCorrectClassified += 1
                        
                else:
                    if(samplePosTest.iloc[i] == 0):
                        totalCorrectClassified += 1
                        
        
        
    elif(typeOfTree == 3):
        for i in range(len(dfTest)):
            if(LikArray[i,1-1] != 0):
                if(LikArray[i,2-1] != 0):
                    if(LikArray[i,3-1] != 0):
                        if(samplePosTest.iloc[i] == 1):
                            totalCorrectClassified += 1
                    else:
                        if(samplePosTest.iloc[i] == 0):
                            totalCorrectClassified += 1
                else:
                    if(LikArray[i,4-1] != 0):
                        if(samplePosTest.iloc[i] == 1):
                            totalCorrectClassified += 1
                    else:
                        if(samplePosTest.iloc[i] == 0):
                            totalCorrectClassified += 1
            else:
                if(LikArray[i,5-1] != 0):
                    if(LikArray[i,6-1] != 0):
                        if(samplePosTest.iloc[i] == 1):
                            totalCorrectClassified += 1
                    else:
                        if(samplePosTest.iloc[i] == 0):
                            totalCorrectClassified += 1
                else:
                    if(LikArray[i,7-1] != 0):
                        if(samplePosTest.iloc[i] == 1):
                            totalCorrectClassified += 1
                    else:
                        if(samplePosTest.iloc[i] == 0):
                            totalCorrectClassified += 1
    elif(typeOfTree == 4):
        for i in range(len(dfTest)):
            if(LikArray[i,1-1] == 1):
                if(LikArray[i,2-1] == 1):
                    if(LikArray[i,3-1] == 1):
                        if(LikArray[i,4-1] == 1):
                            if(samplePosTest.iloc[i] == 1):
                                totalCorrectClassified += 1
                        else:
                            if(samplePosTest.iloc[i] == 0):
                                totalCorrectClassified += 1
                    else:
                        if(LikArray[i,5-1] == 1):
                            if(samplePosTest.iloc[i] == 1):
                                totalCorrectClassified += 1
                        else:
                            if(samplePosTest.iloc[i] == 0):
                                totalCorrectClassified += 1
                else:
                    if(LikArray[i,6-1] == 1):
                        if(samplePosTest.iloc[i] == 1):
                            totalCorrectClassified += 1
                    else:
                        if(samplePosTest.iloc[i] == 0):
                            totalCorrectClassified += 1
            else:
                if(LikArray[i,7-1] == 1):
                    if(samplePosTest.iloc[i] == 1):
                        totalCorrectClassified += 1
                else:
                    if(samplePosTest.iloc[i] == 0):
                        totalCorrectClassified += 1
                    
                
    else:
        print('error treetype not found...')
        
    return totalCorrectClassified / samplePosTest.size
    

In [5]:
solTime = 60*3 #2 minutes
solTime = 4 #2 minutes
C = 1  #weight to objective function
ExclSameBranchFollowing = False #constraint (9) and (10)
numerical = False
combCon = False
maxCard = 3
sensOn = False
specOn = False
sensitivity = 0.95
specificity = 0.95
featCART = True

strength = True #constraint (11) and (12)
anchor = True #constraint (13)
relax = True

samplesizes = [600, 1200, 1800, 2400]
datasets = ['adult', 'chess', 'mushroom', 'heloc']

typeOfTrees = [1, 2, 3, 4]

for typeOfTree in typeOfTrees:
    for dataset in datasets:
        for samplesize in samplesizes:
            listSolTime = []
            listTrainAcc = []
            listTestAcc = []
            for i in range(1, 6):
                seed = i

                DataPreperation(dataset, samplesize, featCART, seed)

                model = decisionTree(typeOfTree, C, ExclSameBranchFollowing, strength, anchor, relax, numerical, combCon, maxCard, sensOn, specOn, sensitivity, specificity)
                model.set_time_limit(solTime)

                sol = model.solve()

                objValue = sol.get_objective_value()
                listSolTime.append(model.solve_details.time)
                listTrainAcc.append( objValue / len(samplePos))
                validation = treeValidation(sol, testDataDummies, trainDataDummies, typeOfTree)
                listTestAcc.append(validation)

            meanSolTime = sum(listSolTime) / len(listSolTime)
            meanTrainAcc = sum(listTrainAcc) / len(listTrainAcc)
            meanTestAcc = sum(listTestAcc) / len(listTestAcc)
            print(dataset +' ' + str(samplesize) +' '+ str(typeOfTree) +' results: '+ str(meanTrainAcc*100) + ' (' + str(meanTestAcc*100) + ')' + '/' + str(meanSolTime))



adult 600 1 results: 83.09999999999998 (80.2189054726368)/4.018399999989197
adult 1200 1 results: 80.41666666666666 (78.12345679012346)/4.068400000006659
adult 1800 1 results: 72.71468144044321 (71.05590062111801)/4.034600000001956
adult 2400 1 results: 72.71468144044321 (71.05590062111801)/4.024999999994179
chess 600 1 results: 86.96666666666665 (86.90558766859343)/1.178000000002794
chess 1200 1 results: 87.03333333333333 (86.84711779448622)/3.684399999998277
chess 1800 1 results: 69.61111111111111 (69.27598566308244)/4.359199999988777
chess 2400 1 results: 59.49166666666667 (59.34591194968554)/4.415599999995902
mushroom 600 1 results: 99.46666666666667 (99.40457203615098)/0.36539999999804423
mushroom 1200 1 results: 99.50000000000001 (99.39341421143848)/1.1468000000051688
mushroom 1800 1 results: 99.45555555555556 (99.39595192915876)/2.5123999999952504
mushroom 2400 1 results: 98.11666666666665 (98.0503144654088)/3.6752000000036786
heloc 600 1 results: 72.60000000000001 (69.376204483

In [6]:
print(sol)

solution for: tree
objective: 1425
V_1_1=1.000
V_1_2=1.000
V_1_3=1.000
V_1_5=1.000
V_1_7=1.000
V_2_4=1.000
V_2_6=1.000
Z_7_5=1.000
Z_7_7=1.000
Z_8_5=1.000
Z_8_7=1.000
Z_9_5=1.000
Z_9_7=1.000
Z_10_5=1.000
Z_10_7=1.000
Z_13_4=1.000
Z_13_6=1.000
Z_14_4=1.000
Z_14_6=1.000
Z_15_4=1.000
Z_15_6=1.000
Z_16_4=1.000
Z_16_6=1.000
Z_1_1=1
Z_1_2=1
Z_1_3=1
Z_2_1=1
Z_2_2=1
Z_2_3=1
Z_3_1=1
Z_3_2=1
Z_3_3=1
Z_4_1=1
Z_4_2=1
Z_4_3=1
Z_5_2=1
Z_5_3=1
Z_6_2=1
Z_6_3=1
Z_7_1=1
Z_7_2=1
Z_7_3=1
Z_8_1=1
Z_8_2=1
Z_8_3=1
Z_9_1=1
Z_9_2=1
Z_9_3=1
Z_10_1=1
Z_10_2=1
Z_10_3=1
C_1_14=1.000
C_1_16=1.000
C_1_21=1.000
C_1_26=1.000
C_1_29=1.000
C_1_30=1.000
C_1_39=1.000
C_1_40=1.000
C_1_44=1.000
C_1_45=1.000
C_1_57=1.000
C_1_61=1.000
C_1_63=1.000
C_1_66=1.000
C_1_77=1.000
C_1_78=1.000
C_1_80=1.000
C_1_81=1.000
C_1_89=1.000
C_1_94=1.000
C_1_95=1.000
C_1_97=1.000
C_1_98=1.000
C_1_101=1.000
C_1_103=1.000
C_1_107=1.000
C_1_114=1.000
C_1_119=1.000
C_1_121=1.000
C_1_123=1.000
C_1_124=1.000
C_1_127=1.000
C_1_133=1.000
C_1_137=1.000